# Exploratory Notebook for bits & bobs

In [6]:
# Standard imports

import numpy as np
import pandas as pd
import json
import os

import matplotlib.pyplot as plt
import seaborn as snsb


## Intersection over Union

In [79]:
def return_IoU(box1, box2):

    """Function to generate the IoU for two bounding boxes in COCO format

    box1: list of floats representing bounding box (COCO format)
    box2: list of floats representing bounding box (COCO format)
    """

    # Convert from COCO format to corner format, this makes it simpler to calculate IoU
    box1 = [box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]]
    box2 = [box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]]

    # Calculate intersection coordinates
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    # Check if there is no overlap
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate intersection area
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou


In [73]:
# Get data path

currend_wd = os.getcwd()
target_file = "annotations.json"

annotations_json = os.path.join(currend_wd, '..', 'raw_data', target_file)

# Load json file

with open(annotations_json, 'r') as file:
    data = json.load(file)


In [74]:
data.keys()


dict_keys(['images', 'annotations', 'info', 'licenses', 'categories'])

In [75]:
subset_annotations = data['annotations'][:5]
test_bbox_0 = subset_annotations[0]['bbox']
test_bbox_1 = subset_annotations[1]['bbox']
test_bbox_2 = subset_annotations[2]['bbox']


test_bbox_0, test_bbox_1


([439.01, 187.01, 56.98000000000002, 53.98000000000002],
 [142.01, 185.01, 51.98000000000002, 54.98000000000002])

In [80]:
display(return_IoU(test_bbox_0, test_bbox_1))

# display(return_IoU(test_bbox_0, test_bbox_2))

# display(return_IoU(test_bbox_1, test_bbox_2))


0.0